In [5]:
# Instalación de Selenium y Chromedriver
import warnings
warnings.filterwarnings('ignore')

!pip install Selenium #instalar selenium 
!apt-get update #para contar con los últimos paquetes disponibles
!apt install chromium-chromedriver # paquete que permite emular google chrome

import sys 
sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver') # el chrome driver es guardado en este link
print('==> Setting up environment - Done')


zsh:1: command not found: apt-get
The operation couldn’t be completed. Unable to locate a Java Runtime.
Please visit http://www.java.com for information on installing Java.

==> Setting up environment - Done


In [10]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException


# Generamos el webdriver para Chrome e indicamos el url de donde extraeremos la información
chrome_driver_path = '/path/to/chromedriver'  # Path to the chromedriver executable
service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=service)

url = 'https://venezuela360.org/arcgis/apps/sites/#/infraestructura-geoespacial/datasets/3d0308d198ba422fbdabd2e1859eb577/data?geometry=-103.593%2C-0.931%2C-29.765%2C14.299'

# Indicamos que se abra la página y espere que cargue
driver.get(url)
wait = WebDriverWait(driver, 10)

# Definimos la función que indica cómo se deben extraer los datos
def extract_table_data(table):
    # Initialize a new dataframe for each table
    table_data_frame = pd.DataFrame()

    # Los nombres de las columnas de nuestro Dataframe deberán ser extraídos de la primera fila
    column_names = [th.text.strip() for th in table.find_elements(By.TAG_NAME, 'th')]

    # Para las demás filas, indicamos que extraigan y se agreguen a nuestro Dataframe
    rows = table.find_elements(By.TAG_NAME, 'tr')
    for row in rows[1:]:
        # Extraemos los valores para cada fila
        values = [td.text.strip() for td in row.find_elements(By.TAG_NAME, 'td')]
        # Creamos un diccionario con los nombres de las columnas como llaves y los valores 
        row_data = {column: value for column, value in zip(column_names, values)}
        # Usamos append para agregar los valores al dataframe
        table_data_frame = table_data_frame.append(row_data, ignore_index=True)

    return table_data_frame


# Generamos el dataframe donde se colocaran los valores extraidos
data_frame = pd.DataFrame()

# Definimos la presencia del botón para pasar a la siguiente tabla e indicamos que el programa automático haga click
next_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="ember114"]/nav/ul/li[13]/a')))

while True:
    try:
        # Esperamos hasta que se pueda identificar la nueva tabla
        wait.until(EC.presence_of_element_located((By.TAG_NAME, 'table')))
        
        # Indicamos que se encuentren todas las tablas en la página
        tables = driver.find_elements(By.TAG_NAME, 'table')

        # Para cada tabla encontrada, aplicamos la función definida anteriormente
        for table in tables:
            table_data = extract_table_data(table)
            data_frame = data_frame.append(table_data, ignore_index=True)

        # Indicamos que se encuentre el botón para pasar a la siguiente tabla
        next_button = driver.find_element(By.XPATH, '//*[@id="ember114"]/nav/ul/li[13]/a')
        
        if not next_button.is_enabled():
            break
            
        next_button.click()
    except StaleElementReferenceException:
        # Esta línea permite que se vuelva a buscar el botón para pasar a la siguiente tabla en caso se encuentre con un "stale element"
        next_button = driver.find_element(By.XPATH, '//*[@id="ember114"]/nav/ul/li[13]/a')
        
    except NoSuchElementException:
        # Esta línea indica que ante el problema de "no such element exception", el programa debe entender que ya no hay mas tables y debe parar. 
        break
        
# Detenemos el navegador
driver.quit()

# Demostramos un preview de los datos y exportamos nuestra base de datos lista
print(data_frame)
data_frame.to_csv('resultado_scrapping_grupo7.csv', index=False)


      Abstencion2000 Abstencion2001 Abstencion2006 Abstencion2007  \
0                  0            927            650            965   
1                  0           1144            546            891   
2                  0           1217            650            878   
3                  0           1556            512            718   
4                  0           1904            867           1254   
...              ...            ...            ...            ...   
13158            155            187              0              0   
13159            528            716              0              0   
13160            473            445            253            354   
13161           1167           1000            897           1493   
13162            840            758            400            705   

      Abstencion2008 Abstencion2009 Abstencion2010 Abstencion2012  \
0                999            660            821            474   
1               1060            7